In [96]:
# add autoreload
%load_ext autoreload
%autoreload 2
import sickle
from tqdm import tqdm
import os

import requests
import lxml
import bs4
import random
from time import sleep

import oai

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [97]:
source = 'UVA'

In [145]:
accepted_topiclist = ['athero', 
                        'plaque', 
                        'cardiovascular',
                        'cardiogram', 
                        'cardiology', 
                        'cardiologie',
                        'hartvaten',
                        'klinsch',
                        'medische',
                        'hartvaat',
                        'heart', 
                        'vascular',
                        'angiogram', 
                        'cardiologie', 
                        'hartziekte', 
                        'vaatziekte',
                        'medicine',
                        'disease', 
                        'medical', 
                        'therapy',
                        'therapeutic',
                        'diagnosic',
                        'clinical',
                        'surgical', 
                        'metabolic',
                        'myocard']

accepted_topiclist = ['recht', 'wetten', 'juridisch',
                        'rechtspraak', 'verordering', 'wetgeving',
                        'richtlijn',
                        'regelgeving',
                        'wetboek',
                        'wetboek', 'jurisprudentie', 
                        'precedent', 'wet bibop',
                        'wetboek van strafrecht',
                        'wetboek van strafvordering']

accepted_dtypes = ['doctoral', 'book', 'article']
accepted_languages = ['nl','nld','dut', 'und'] 


base_url =   oai.sources[source]['link'] #'https://repository.ubn.ru.nl/oai/openaire'  https://scholarlypublications.universiteitleiden.nl/oai2, http://dspace.library.uu.nl/oai/dissertation
pdf_path = f'//Ds/data/LAB/laupodteam/AIOS/Bram/language_modeling/MEDICAL_TEXT/RAW/PhDTheses/{source}/'
pdf_path = f'/media/koekiemonster/DATA-FAST/text_data/pubscience/PDF/LAW/RUG' 

In [99]:
OpenAIRE_institutes = ['VU', 'UVA', 'Maastricht',
                       'Tilburg', 'RUG', 'UTwente', 
                       'TUE', 'UU', 'Erasmus']

In [100]:
sickler = sickle.Sickle(base_url)

In [101]:
sets = sickler.ListSets()

In [102]:
Sets = {}
for s in sets:
    Sets[s.setSpec]  = s.setName    

In [103]:
keywords = ['clinical', 'medisch', 'medical', 'dissertation', 'umc', 'medicine',
            'diss', 'phd', 'thesis', 'doctorate', 'dissertatie',
            'doctoraat', 'proefschrift']
if source in OpenAIRE_institutes:
    keywords = keywords + ['publications:withfiles']

Sets_to_mine = []
for key, val in Sets.items():
    #print(key,val)
    if any([c in val.lower() for c in keywords]) | any([c in key.lower() for c in keywords]):
        print(f'Set: {key} contains keyword')
        Sets_to_mine.append(key)

Set: publications:withFiles contains keyword


In [104]:
# get records 
# Beware: this takes a long time.
from collections import defaultdict
records_lists = defaultdict(list)
for set_to_mine in Sets_to_mine:
    #if set_to_mine in ['com_1874_298213']:
    #    continue
    print(f"Mining from set: {set_to_mine}")
    try:
        records = sickler.ListRecords(metadataPrefix='oai_dc', 
                                    ignore_deleted=True, 
                                    set=set_to_mine) # dissertation com_1874_298213
        for record in tqdm(records):
            records_lists[set_to_mine].append(record)
    except Exception as e:
        print(e)
        continue

Mining from set: publications:withFiles


62283it [30:06, 34.48it/s]


In [105]:
records_lists['publications:withFiles'][0].get_metadata()

{'title': ['D4.4 Prototype of the system for enhanced services recommendation'],
 'creator': ['Sargolzaei, M.', 'Shafahi, M.', 'Afsarmanesh, H.'],
 'description': ['This deliverable addresses the prototypes of two tools that are implemented as a part of the PSS sub-system in the GloNet system, namely the Service Specification Tool (SST) and Product/Service Discovery and Recommendation (PSDR) tool. The designs of these tools are represented in the deliverable D4.3. These two tools, as well as the Product Specification Tool (PST) represented in D4.1 and D4.2, are well integrated as PSS (Product and Service Specification), supporting proper specification of different aspects of the complex products. The PSS sub-system provides the set of needed mechanisms for specification, registration, discovery, and recommended ranking of sub-products and business services, as well as the composition of business services and assisting with the enhancement for products with business services. As such, t

In [ ]:
# TODO ADD SEMANTIC SEARCH FOR LEGAL TEXTS

In [160]:
filtered_records_lists = defaultdict(list)
excluded_records_lists = defaultdict(list)

cond_list = []

RAW_MINE_RESULTS = []

for set_to_mine in Sets_to_mine:
    relevant_counter = 0
    for r in tqdm(records_lists[set_to_mine]):
        meta = r.get_metadata()
        relevant = False
        
        TOPIC = False
        PDF = False
        DOCTORATE = True if 'dissertation' in set_to_mine.lower() else False
        EMBARGO = False
        LANG = False
        
        try:
            _lang = meta['language'][0]
        except KeyError:
            _lang = 'unknown'
            
        if _lang in accepted_languages:
            LANG = True
        

        if source in ['Radboud']: 
            PDF=True
        
        try:
            _subj = ",".join(meta['subject'])
            if any([t in _subj.lower() for t in accepted_topiclist]):
                TOPIC = True
        except:
            _subj = ""
            pass

        try:
            _title = meta['title']
            if any([t in subj.lower() for subj in _title for t in accepted_topiclist]):
                TOPIC = True
        except:
            _title = ""
            pass

        try:
            _description = meta['description']
            if any([t in subj.lower() for subj in _description for t in accepted_topiclist]):
                TOPIC = True
        except:
            _description = ""
            pass                        
        
        try:
            _format = meta['format'][0].lower()
            if ('pdf' in _format):
                PDF = True
        except:
            _format = ""
            pass
        
        try:
            _type = meta['type'][0].lower()
            if any([t in +_type for t in accepted_dtypes]):
                DOCTORATE = True
        except:
            _type = ""
            pass

        try:
            _rights = meta['rights'][0].lower()
            if ('embargo' in _rights) |\
                    ('restricted' in _rights):
                EMBARGO = True
        except:
            _rights = ""
            pass  
        
        try:
            _date = meta['date'][0].lower()
        except:
            _date = ""
            pass
        
        RAW_MINE_RESULTS.append(
                {
                'LANG': _lang,
                'TOPIC': _subj,
                'TITLE': _title,
                'DESCRIPTION': _description,            
                'FORMAT': _format,
                'TYPE': _type,
                'RIGHTS': _rights,
                'DATE': _date,
                'TOPIC_OF_INTEREST': TOPIC,
            }
        )
        
        cond_list.append({'Topic': TOPIC, 
                          'PDF': PDF, 
                          'Doctorate': DOCTORATE, 
                          'Embargo': EMBARGO,
                          'LANG': LANG}
                         )
        
        if TOPIC & PDF & DOCTORATE & LANG & ~EMBARGO:
            relevant_counter += 1
            filtered_records_lists[set_to_mine].append(r)
        else:
            excluded_records_lists[set_to_mine].append(r)

    print(f'Found {relevant_counter} relevant records in set: {set_to_mine}')      

100%|██████████| 62283/62283 [00:09<00:00, 6536.34it/s] 

Found 0 relevant records in set: publications:withFiles


In [170]:
# load SBERT model
from sentence_transformers import SentenceTransformer
SBERT = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')# ('paraphrase-multilingual-mpnet-base-v2')

[autoreload of torch._utils failed: Traceback (most recent call last):
  File "/media/koekiemonster/home/bramiozo/.pyenv/versions/3.10.4/envs/nlp/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/media/koekiemonster/home/bramiozo/.pyenv/versions/3.10.4/envs/nlp/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/media/koekiemonster/home/bramiozo/.pyenv/versions/3.10.4/lib/python3.10/importlib/__init__.py", line 159, in reload
    raise ImportError(msg.format(parent_name),
ImportError: parent 'torch' not in sys.modules
]
[autoreload of torch._utils_internal failed: Traceback (most recent call last):
  File "/media/koekiemonster/home/bramiozo/.pyenv/versions/3.10.4/envs/nlp/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/media/koekiemonster/h

AttributeError: module 'numpy.linalg' has no attribute '_umath_linalg'

In [159]:
sorted(lang_dict.items(), key=lambda x: x[1], reverse=True)[:10]

[('eng', 43509),
 ('und', 10783),
 ('nld', 6897),
 ('deu', 284),
 ('fra', 191),
 ('spa', 191),
 ('ita', 98),
 ('rus', 77),
 ('dan', 45),
 ('por', 39)]

In [107]:
types_counts = defaultdict(int)
for r in excluded_records_lists['publications:withFiles']:
    try:
        types_counts[r.metadata['type'][0]] += 1
    except:
        pass

In [118]:
types_counts

defaultdict(int,
            {'book': 3120,
             'article': 38087,
             'bookPart': 3904,
             'doctoralThesis': 10366,
             'contributionToPeriodical': 3263,
             'workingPaper': 1314,
             'other': 895,
             'conferenceObject': 145})

In [119]:
conds_df = pd.DataFrame(cond_list)

In [120]:
conds_df.sum()

Topic         2015
PDF          60604
Doctorate    56666
Embargo      10549
LANG          6897
dtype: int64

In [121]:
tot_theses_with_pdf = (conds_df.Doctorate & conds_df.PDF).sum()
print(f"Total number of theses with PDF: {tot_theses_with_pdf}")

Total number of theses with PDF: 55094


In [122]:
link_list = []
error_list = []

for set_to_mine in filtered_records_lists.keys():
    for record in tqdm(filtered_records_lists[set_to_mine]):
        META = record.get_metadata()
        try:
            List_of_identifiers = META['identifier'] if 'identifier' in META.keys() else ['']
            Title = META['title'] if 'title' in META.keys() else ['']
            Description = META['description'] if 'description' in META.keys() else ['']
            Date = META['date'] if 'date' in META.keys() else ['']
            Language = META['language'] if 'language' in META.keys() else ['']       
            Creator = META['creator'] if 'creator' in META.keys() else ['']
            
            
            List_of_identifiers = [id for id in List_of_identifiers if id is not None]
            
            if len(List_of_identifiers)>0:
                link_list.append({'Set': set_to_mine, 
                                'Link': List_of_identifiers, 
                                'Title': Title,
                                'Description': Description,
                                'Date': Date ,
                                'Language': Language,
                                'Creator': Creator,
                                'Publisher': META.get('publisher'),
                                }
                                )
            else:
                continue
        except Exception as e:
            error_list.append(f"Error parsing {e}: {META}: ")
            pass

  0%|          | 0/1189 [00:00<?, ?it/s]

100%|██████████| 1189/1189 [00:00<00:00, 12698.17it/s]


In [123]:
publisher_counts = defaultdict(int)
for r in link_list:
    try:
        publisher_counts[r['Publisher']] += 1
    except:
        pass
publisher_counts

defaultdict(int, {None: 1178})

In [124]:
link_list[500]

{'Set': 'publications:withFiles',
 'Link': ['https://dare.uva.nl/personal/pure/en/publications/billijke-vergoeding-in-recht-en-economie(1a2cc12d-81f9-4783-92bc-910f1837ec5a).html',
  '11245/1.501089',
  'https://pure.uva.nl/ws/files/2682758/170178_2015_Billijke_vergoeding_in_recht_en_economie_Poort_AMI_6_2015_p_1_5.pdf'],
 'Title': ['Billijke vergoeding in recht en economie'],
 'Description': ["Op 2 oktober 2013 wees de Rechtbank Den Haag vonnis in de zaak van SENA tegen 25 organisatoren van 'dance events' over de billijke vergoeding die zij aan SENA verschuldigd zijn voor het draaien van muziekopnames. Hoger beroep dient bij het Haagse gerechtshof, dat de zaak naar verwachting voor advies zal verwijzen naar de Geschillencommissie Auteursrecht. Niet eerder liet een Nederlandse rechter zich expliciet uit over de hoogte van de billijke vergoeding en nu de billijke vergoeding voor auteurs ook in het nieuwe auteurscontractenrecht een centrale rol speelt, rijst de vraag hoe zo'n vergoeding 

In [125]:
import re

In [126]:
def get_pdfs_Leiden(url):
    try:
        r = requests.get(url, timeout=10)
        # random number between 0.5 and 2.5 seconds
        rndSleep = round(random.uniform(1, 5), 2)
        sleep(rndSleep)
        soup = bs4.BeautifulSoup(r.text, 'html.parser')
        found, dsfound, esfound = False, False, False
        for _res in soup.findAll('li', {'class':'ubl-file-view'}):
            if _res.a is not None:
                if _res.a.contents[0].strip().lower() == 'full text':
                    _pdfdir = _res.a['href']
                    found = True
                elif _res.a.contents[0].strip().lower() == 'summary in dutch':
                    _dutch_summary = _res.a['href']
                    dsfound = True
                elif _res.a.contents[0].strip().lower() == 'summary in english':
                    _english_summary = _res.a['href']
                    esfound = True

        linkPdfAlt = f"https://scholarlypublications.universiteitleiden.nl{_pdfdir}" if found else None
        DutchSummaryLink = f"https://scholarlypublications.universiteitleiden.nl{_dutch_summary}" if dsfound else None
        EnglishSummaryLink = f"https://scholarlypublications.universiteitleiden.nl{_english_summary}" if esfound else None
        
        return linkPdfAlt, DutchSummaryLink, EnglishSummaryLink, r.status_code
    except Exception as e:
        return None, None, None, None

In [127]:
def extract_pdf_links(links, institute=None):
    pdf_links = []
    if institute in ['VU', 'UVA', 'UTwente']:
        inclusion_terms = [r'abstract', r'full', r'complete', r'samenvatting', r'summary', r'thesis', r'chapter']
    elif institute in ['Radboud']:
        inclusion_terms = [r'handle', r'bitstream']
    elif institute in ['Maastricht']:
        inclusion_terms = [r'ASSET1', r'c[0-9]{3,4}\.pdf']
    elif institute in ['Tilburg']:
        inclusion_terms = [r'\.pdf']
    elif institute in ['RUG']:
        inclusion_terms = [r'summ\.pdf', r'summary\.pdf',  r'samenv\.pdf', r'samenvat\.pdf', r'[ch][0-9]{1,2}\.pdf', 
                           r'thesis\.pdf', r'proefschrift\.pdf', r'dissertation\.pdf', r'dissertatie\.pdf']
    elif institute in ['TUE']:
        inclusion_terms = [r'summ\.pdf', r'summary\.pdf',  r'samenv\.pdf', r'samenvat\.pdf', r'[ch][0-9]{1,2}\.pdf', 
                           r'thesis\.pdf', r'proefschrift\.pdf', r'dissertation\.pdf', r'dissertatie\.pdf']
        inclusion_terms = inclusion_terms + [r'abstract', r'full', r'complete', r'samenvatting', r'summary', r'thesis', r'chapter']
    elif institute in ['UU']:
        inclusion_terms = [r'dspace\.library\.uu\.nl']
    elif institute in ['Leiden']:
        inclusion_terms = [r'handle']
    elif institute in ['Erasmus']:
        inclusion_terms = [r'files']
    else:
        raise ValueError(f'Institute {institute} not recognized')
    
    inclusion_terms = [re.compile(rs) for rs in inclusion_terms]
    
    if institute in ['UU']:
        _pdf_links = []
        for link in links:
            if (link.lower().startswith('http')):
                baselink = link.replace('dspace.library.uu.nl/', 'dspace.library.uu.nl/bitstream/')
                _pdf_links.append(baselink + '/full.pdf')
        pdf_links = _pdf_links
    elif institute in ['Leiden']:
        _pdf_links = []
        for link in links:
            if (link.lower().startswith('http')) & (any([t.search(link) is not None for t in inclusion_terms])):
                main, dutch_summ, engl_summ, return_code = get_pdfs_Leiden(link)
                if return_code == 429:
                    print("Too many requests. Waiting 60 seconds")
                    sleep(60)
                else:
                    _pdf_links.extend([main, dutch_summ, engl_summ])
        pdf_links = [l for l in _pdf_links if l is not None]
    else:
        for link in links:
            if (link.lower().startswith('http')) & \
                    (link.lower().endswith('.pdf')):
                if any([t.search(link) is not None for t in inclusion_terms]):
                    pdf_links.append(link)        

    return pdf_links

In [128]:
len(link_list)

1189

In [129]:
link_pdf_list = []
for l in tqdm(link_list):
    links = extract_pdf_links(l['Link'], institute=source)
    tmp = []
    for _l in links:
        Creator = l['Creator'][0] if l['Creator'][0] is not None else 'Unknown'
        Date = l['Date'][0] if l['Date'][0] is not None else 'Unknown'
        
        pdfPath = _l.split("/")[-1].replace("%20", "_").rstrip('.pdf')
        pdfPath = source + "_" + Creator + "_" + Date + "_" + pdfPath
        pdfPath = pdfPath.replace(",", "")
        pdfPath = pdfPath.replace(":", "")
        pdfPath = pdfPath.replace(".", "")
        pdfPath = pdfPath.replace(" ", "")
        pdfPath = os.path.join(pdf_path, pdfPath+".pdf")
        tmp.append((_l, pdfPath))
    if len(tmp)==0:
        continue
    l['pdf_links'] = tmp
    link_pdf_list.append(l)

100%|██████████| 1189/1189 [00:00<00:00, 37966.89it/s]


In [130]:
len(link_pdf_list)

47

In [132]:
# remove duplicate entries, based on the title
title_set = set()
unique_link_list = []
for el in link_pdf_list:
    try:
        title = el['Title'][0]
        if title not in title_set:
            unique_link_list.append(el)
            title_set.add(title)
    except Exception as e:
        pass

In [133]:
len(unique_link_list)

47

In [134]:
# now we parse the link list and download the pdfs
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1700.107 Safari/537.36' }

# add sleep
def pdf_writer(pdf_url, _pdf_path):
    r = requests.get(pdf_url, stream=True)
    if r.status_code == 200:
        with open(_pdf_path, 'wb') as f:
            for i,chunk in enumerate(r.iter_content(chunk_size=1024)): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
        return True, 200
    else:
        return False, r.status_code

In [95]:
if source=='Radboud':
    filelist = os.listdir(pdf_path)
    pre_list = [f.split("_")[-1] for f in filelist]
if source=='UU':
    filelist = os.listdir(pdf_path)
    pre_list = [f.split(".")[-2] for f in filelist]

pdf_error_list = []
skipped_list = []
src_list = []
rcode_list = []
success_list = []
extract_full_text = True
for link in tqdm(unique_link_list):
    for lt in link['pdf_links']:
        _pdf_path = lt[1]
        pdf_url = lt[0]
        return_code = None

        if source=='Radboud':
            if _pdf_path.split("_")[-1] in pre_list:
                skipped_list.append(f'Pdf already exists: {_pdf_path}')
                continue
        if source=='UU':
            if link['pdf_links'][0][0].split("/")[-2] in pre_list:
                skipped_list.append(f'Pdf already exists: {_pdf_path}')
                continue
            
        if os.path.isfile(_pdf_path):
            skipped_list.append(f'Pdf already exists: {_pdf_path}')
        else:
            # try to download the pdf
            try:
                write_file, return_code = pdf_writer(pdf_url, _pdf_path)
                success_list.append(f'Pdf downloaded: {_pdf_path}')
            except Exception as e:
                pdf_error_list.append(f'Error: {e} for link: {link["Link"]}')  

            if return_code == 429:
                print(f'Rate limit exceeded...sleeping 30 seconds')
                sleep(30)
                continue
            else:
                sleep(4)


  0%|          | 0/52 [00:00<?, ?it/s]

100%|██████████| 52/52 [07:43<00:00,  8.91s/it]


In [ ]:
print(f'Found {len(pdf_error_list)} errors while downloading pdfs')
print(f'Skipped {len(skipped_list)} pdfs because they already existed')

Found 0 errors while downloading pdfs
Skipped 1 pdfs because they already existed
